In [ ]:
import pandas as pd
from pathlib import Path
import re
import hashlib

# ============================================================
# 1) Load the dictionary (CSV)
# ============================================================
# Put the real CSV path in your environment.
# If it's in the same folder as your notebook:
# "MasterMap_NOTES_ES_EN.csv"
csv_path = "MasterMap_NOTES_ES_EN.csv"
MAP_PATH = Path(csv_path).resolve()
OUT_DIR = MAP_PATH.parent

# Read CSV into a DataFrame
df = pd.read_csv(csv_path)

# Normalize column names in case there are weird leading/trailing spaces
df.columns = [c.strip() for c in df.columns]

# Expected columns (adjust if your CSV uses slightly different names)
COL_SERVICE = "Servicio AWS"
COLS_OUT = ["Azure", "GCP", "Open Source", "Quality", "Notes_EN"]

# Validate required columns exist
missing_cols = [c for c in [COL_SERVICE] + COLS_OUT if c not in df.columns]
if missing_cols:
    raise ValueError(
        f"Missing columns in the CSV: {missing_cols}. Columns present: {list(df.columns)}"
    )

# Trim whitespace from service names (defensive cleaning)
df[COL_SERVICE] = df[COL_SERVICE].astype(str).str.strip()

# Build an exact-match index: { "S3": row_index, ... }
index_exact = {s: i for i, s in enumerate(df[COL_SERVICE].tolist())}

# Build a case-insensitive index (e.g., user types "s3" instead of "S3")
index_lower = {}
for i, s in enumerate(df[COL_SERVICE].tolist()):
    key = s.lower()
    # If multiple services collide by case, keep the first occurrence
    index_lower.setdefault(key, i)

def _sanitize_filename_part(s: str) -> str:
    s = s.strip()
    s = re.sub(r"\s+", "_", s)
    s = re.sub(r"[^A-Za-z0-9_\-]", "-", s)  # seguro para Windows/Drive
    s = re.sub(r"-{2,}", "-", s)
    s = re.sub(r"_{2,}", "_", s)
    return s.strip("._-")

def _make_output_name(items: list[str], max_len: int = 140) -> str:
    parts = [_sanitize_filename_part(x) for x in items if x.strip()]
    base = "_".join([p for p in parts if p]) or "lookup_result"
    if len(base) <= max_len:
        return base + ".csv"
    h = hashlib.md5(base.encode("utf-8")).hexdigest()[:8]
    base_short = base[:max_len].rstrip("._-")
    return f"{base_short}_{h}.csv"

def lookup_services(services_csv: str, *, show_messages: bool = True) -> pd.DataFrame:
    """
    services_csv: A string with one or more service names separated by commas.
                  Example: "S3, UserCompanyDeveloper, EKS, ECR, EC2, AMI"

    Returns a DataFrame with columns:
      Servicio AWS, Azure, GCP, Open Source, Quality, Notes_EN, Status

    Special rules:
      - UserCompany*/UserConsumer* => not cloud services: returns NA + a default note
      - Not found in the dictionary => Status = NOT_FOUND + a message to review the name
    """
    if services_csv is None:
        services_csv = ""

    # --- Default note for actors/endpoints (not cloud services) ---
    USER_DEFAULT_NOTE_EN = (
        "This entry is not a cloud service. 'UserCompany*' and 'UserConsumer*' represent end-user actors, "
        "roles, channels, or endpoints (e.g., personas, devices, client apps). They should not be mapped to "
        "Azure/GCP/OSS service equivalents."
    )

    # Explicit actor codes (plus prefix-based checks below)
    USER_ACTOR_CODES = {
        "UserCompanyAgent",
        "UserCompanyAnalyst",
        "UserCompanyAPI",
        "UserCompanyCRM",
        "UserCompanyDataStream",
        "UserCompanyDeveloper",
        "UserCompanyDomainExpert",
        "UserCompanyDrone",
        "UserCompanyEdge",
        "UserCompanyElementalLiveDevice",
        "UserCompanyHeadEnd",
        "UserCompanyInternalPlatform",
        "UserCompanyWebsite",
        "UserConsumerAlexaGoogleHome",
        "UserConsumerAPI",
        "UserConsumerArtist",
        "UserConsumerCamera",
        "UserConsumerDeveloper",
        "UserConsumerEdge",
        "UserConsumerFarmer",
        "UserConsumerHospital",
        "UserConsumerIOT",
        "UserConsumerMobile",
        "UserConsumerPOS",
        "UserConsumerSatellite",
        "UserConsumerTV",
        "UserConsumerWeb",
        "UserConsumerWebMobile",
    }

    def is_user_actor(name: str) -> bool:
        """
        Returns True if the name corresponds to an end-user actor/endpoint
        rather than a cloud service.
        """
        n = name.strip()
        return (
            n in USER_ACTOR_CODES
            or n.startswith("UserCompany")
            or n.startswith("UserConsumer")
        )

    # Parse comma-separated list -> cleaned items
    items = [x.strip() for x in str(services_csv).split(",")]
    items = [x for x in items if x]  # remove empty entries

    results = []
    not_found = []

    for s in items:
        # 0) Actor/endpoint: return a default "NA" row
        if is_user_actor(s):
            results.append(
                {
                    "Servicio AWS": s,
                    "Azure": "-",
                    "GCP": "-",
                    "Open Source": "-",
                    "Quality": "NA",
                    "Notes_EN": USER_DEFAULT_NOTE_EN,
                    "Status": "USER_ACTOR",
                }
            )
            continue

        row_idx = None

        # 1) Exact match
        if s in index_exact:
            row_idx = index_exact[s]
        else:
            # 2) Case-insensitive match
            row_idx = index_lower.get(s.lower())

        # If still not found, mark as NOT_FOUND
        if row_idx is None:
            not_found.append(s)
            results.append(
                {
                    "Servicio AWS": s,
                    "Azure": "",
                    "GCP": "",
                    "Open Source": "",
                    "Quality": "",
                    "Notes_EN": "",
                    "Status": "NOT_FOUND",
                }
            )
        else:
            # Extract the mapped equivalents from the dictionary row
            r = df.loc[row_idx, [COL_SERVICE] + COLS_OUT].to_dict()
            results.append(
                {
                    "Servicio AWS": r[COL_SERVICE],
                    "Azure": r["Azure"],
                    "GCP": r["GCP"],
                    "Open Source": r["Open Source"],
                    "Quality": r["Quality"],
                    "Notes_EN": r["Notes_EN"],
                    "Status": "OK",
                }
            )

    # Print messages only for NOT_FOUND (not for USER_ACTOR)
    if show_messages and not_found:
        print("Services not found (check the exact spelling as it appears in the dictionary):")
        for s in not_found:
            print(" -", s)

    # Return a consistent column order

    df_res = pd.DataFrame(results, columns=["Servicio AWS"] + COLS_OUT + ["Status"])

    out_name = _make_output_name(items)
    out_path = OUT_DIR / out_name
    df_res.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f" Saved: {out_path}")

    return df_res



In [ ]:
#=== 2) Usage Examples ===
df_out = lookup_services("S3")
df_out = lookup_services("S3, UserCompanyDeveloper, EKS, ECR, EC2, AMI")
display(df_out)
print(df_out.to_string(index=False)) #console

 Saved: /content/S3.csv
 Saved: /content/S3_UserCompanyDeveloper_EKS_ECR_EC2_AMI.csv


,Servicio AWS,Azure,GCP,Open Source,Quality,Notes_EN,Status
0,S3,Blob Storage,Cloud Storage,MinIO,CLOSE,Object storage. Close equivalent (APIs/semanti...,OK
1,UserCompanyDeveloper,-,-,-,NA,This entry is not a cloud service. 'UserCompan...,USER_ACTOR
2,EKS,AKS,GKE,Kubernetes,CLOSE,Managed Kubernetes. Close equivalent; differs ...,OK
3,ECR,Container Registry,Artifact Registry,Harbor,CLOSE,OCI container image registry. Close equivalent...,OK
4,EC2,Virtual Machines,Compute Engine,KVM / OpenStack,CLOSE,VM/IaaS. Close equivalence (instances). Differ...,OK
5,AMI,Managed Image,Compute Engine Image,Packer,CLOSE,Conceptually equivalent to a VM golden image. ...,OK


        Servicio AWS              Azure                  GCP     Open Source Quality                                                                                                                                                                                                                               Notes_EN     Status
                  S3       Blob Storage        Cloud Storage           MinIO   CLOSE                                                                                            Object storage. Close equivalent (APIs/semantics differ: consistency, IAM/policies, storage classes). Common OSS options: MinIO / Ceph RGW.         OK
UserCompanyDeveloper                  -                    -               -      NA This entry is not a cloud service. 'UserCompany*' and 'UserConsumer*' represent end-user actors, roles, channels, or endpoints (e.g., personas, devices, client apps). They should not be mapped to Azure/GCP/OSS service equivalents. USER_ACTOR
                 EK

In [ ]:
df_out = lookup_services("Lambda")
display(df_out)

df_out = lookup_services("Cloudfront")
display(df_out)
#print(df_out.to_string(index=False)) #console

 Saved: /content/Lambda.csv


,Servicio AWS,Azure,GCP,Open Source,Quality,Notes_EN,Status
0,Lambda,Azure Functions,Cloud Functions,OpenFaaS / Knative,CLOSE,FaaS (serverless). Teaches ephemeral execution...,OK


 Saved: /content/Cloudfront.csv


,Servicio AWS,Azure,GCP,Open Source,Quality,Notes_EN,Status
0,CloudFront,Azure Front Door / CDN,Cloud CDN,Varnish / Nginx,CLOSE,Global CDN. Cache invalidation semantics diffe...,OK
